## Recommandation hybride : Content based et Popularity based

On utilise ici une méthode de recommandation basée sur le contenu, à savoir les tags que les utilisateurs ont associés à chaque livre, ainsi qu'une méthode de recommandation basée sur la popularité de chaque livre.

Au départ un livre est renseigné, par exemple le dernier que l'utilisateur a lu. L'algorithme Content based trouve alors 50 livres qui lui ressemblent (de même genre, qui traîte les mêmes sujets), puis l'agorithme popularity based va faire remonter, parmi ces 50 livres, les 10 plus populaires (un livre est populaire si un grand nombre de personnes lui ont attribué une bonne note).

In [12]:
import pandas as pd
import numpy as np

cos_sim = np.load("data/cos_sim.npy")

index_books = pd.read_csv("data/index_books.csv")
books = pd.read_csv("data/books.csv")
popularity_books = pd.read_csv("data/popularity_books.csv")


In [75]:
def get_book_id(title): 
    # Récupère l'ID du livre en fonction de son titre
    return books["goodreads_book_id"][books["title"]==title].item()

def get_book_title(goodreads_id):
    # Récupère le titre du livre en fonction de son id
    return books["title"][books["goodreads_book_id"]==goodreads_id].item()


def get_book_weighted_rating(goodreads_id):
    try:
        return popularity_books["weighted_rating"][popularity_books["goodreads_book_id"]==goodreads_id].item()
    except:
        return 0


def recommendations(title, cos_sim = cos_sim):
    """
    Fonction recommandation :
    Prend en entrée le title d'un livre et la matrice de similarité.
    Trouve l'id du livre
    Cherche dans la matrice les autres livres similaires
    """
    
    # Initialise la liste de livres recommandés
    rec_books_ids = []

    # ID du livre
    book_id = get_book_id(title)
    
    # Index du livre dans la matrice
    idx = index_books[index_books["book"]==book_id].index[0]

    # Création de la série qui comprend la similiraté de notre livre et tous les autres (correspond à sa colonne de la matrice cos_sim)
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # On récupère les 10 premiers indices (plus grandes similarités)
    rec_books_indexes = list(score_series.iloc[1:31].index)
    
    rec_books_ids = list(map(lambda x: index_books["book"][x],rec_books_indexes))

    rec_books_wr = list(map(get_book_weighted_rating,rec_books_ids))


    rec_books_df = pd.DataFrame(list(zip(rec_books_ids, rec_books_wr)),columns=['id','wr'])

    rec_books_df = rec_books_df.sort_values("wr",ascending=False)

    rec_books = list(map(get_book_title, rec_books_df["id"][0:10]))

    return [rec_books, list(rec_books_df["wr"][0:10])]

In [76]:
recommendations("Candide")

[['Les Misérables',
  'The Idiot',
  'The Three Musketeers',
  'The Fall',
  'Song of Solomon',
  'Siddhartha',
  'The Plague',
  'The Hunchback of Notre-Dame',
  'Native Son',
  'The Stranger'],
 [4.129796312304483,
  4.116906087259228,
  4.0499577417714825,
  4.026723569847905,
  4.006819875294491,
  3.9912113764735775,
  3.979656488500729,
  3.978213276934455,
  3.976150383165109,
  3.963751922640032]]